In [1]:
import spacy
from xml.etree import ElementTree
import pickle

In [2]:
dictSpeakerSents = {}
tree = ElementTree.parse('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-sv.xml')
root = tree.getroot()
for speaker in root.iter('SPEAKER'):
    for sent in speaker.iter('s'):
        dictSpeakerSents[sent.attrib['id']] = speaker.attrib['NAME'].replace(' ', '_')



In [3]:
dictSpeakerLines = {}
tree = ElementTree.parse('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-links.xml')
root = tree.getroot()
lineNumber = 0

for link in root.iter('link'):
    sweLinks = link.attrib['xtargets'].split(';')[1].split()
    noSpeakerInLine = False

    for sentNumber in sweLinks:
        if sentNumber not in dictSpeakerSents:
            noSpeakerInLine = True

    for sentNumber in sweLinks:
        if not noSpeakerInLine:
            dictSpeakerLines[lineNumber] = dictSpeakerSents[sentNumber]

    lineNumber += 1


In [4]:
nlp = spacy.load('../Models/SwedishModel', disable=['tagger', 'parser', 'ner'])

In [5]:
dictSpeakerTokens = {}
tokenIndex = 0
with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-sv.txt') as sweDoc:
    for lineIndex, line in enumerate(sweDoc):
        doc = nlp(line)
        if lineIndex in dictSpeakerLines:
            for i, token in enumerate(doc):
                dictSpeakerTokens[tokenIndex + i] = dictSpeakerLines[lineIndex]
        else:
            for i, token in enumerate(doc):
                dictSpeakerTokens[tokenIndex + i] = None
        tokenIndex += len(doc)



In [3]:
sweCorp = ''

with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-sv.txt') as sweDoc:
    for line in sweDoc:
        sweCorp = sweCorp + line

doc = nlp(sweCorp)
ex = len(doc)

In [11]:
assert (tokenIndex == ex), "Number of Tokens not matching"

In [6]:
with open('speakersTokObject', 'wb') as f:
    pickle.dump(dictSpeakerTokens, f)
